# Replay Attack Gas Cost Comparison
---
Here in this notebook, we provided the detailed gas cost breakdown for each scheme.

After investigating the smart contracts provided in the `contracts` directory using Remix IDE, we recorded the remaining gas values at the beginning of each important section in the Solidity files to calculate the cost of each replay attack prevention scheme. 

We referred the following table as the Remaining Gas Table.

***Remaining Gas Table***

|**Section**                  |**AnonParking**|**HashAuth**|**PseudoAuth**|**NPAuth**|
|:---------------------------:|:-------------:|:----------:|:------------:|:--------:|
|Initialization              |62615          |56752       |32352         |30602     |
|Beginning                    |37967          |32616       |8216          |6098      |
|`authenticate` Function      |37848          |32519       |8119          |6001      |
|`checkReplayAttack` Function |37638          |32309       |7909          |5791      |
|`verifyProof` Function       |9572           |8807        |5624          |5396      |
|`emit` Function                |9339           |8574        |5391          |5163      |
|End                          |8168           |7403        |4220          |3992      |

We provided the CSV file for the previous table in this directory.

## Calculating the Operation Cost Table
First of all, we import and transpose the data for easier manipulation inside pandas.

In [ ]:
import pandas as pd
remaining_gas_table=pd.read_csv("remaining_gas_table.csv")
remaining_gas_table_t = remaining_gas_table.transpose()
remaining_gas_table_t.columns = remaining_gas_table_t.iloc[0]
remaining_gas = remaining_gas_table_t[1:]
remaining_gas.columns.name = 'Scheme'
remaining_gas

After getting the previous table, we calculate the operation cost by calculating the difference of the remaining gas from the function we want to investigate. 

In [ ]:
prior_authentication_cost = remaining_gas["beginning"]-remaining_gas["authenticate_function"]
authentication_parameter_cost = remaining_gas["authenticate_function"]-remaining_gas["checkReplayAttack_function"]
check_replay_attack_cost = remaining_gas["checkReplayAttack_function"]-remaining_gas["verifyProof_function"]
verification_cost = remaining_gas["verifyProof_function"]-remaining_gas["emit_function"]
emit_cost = remaining_gas["emit_function"]-remaining_gas["end"]
operation_cost_table = pd.concat([
    prior_authentication_cost,
    authentication_parameter_cost,
    check_replay_attack_cost,
    verification_cost,
    emit_cost
], axis=1, keys=['prior_authentication','authentication_parameter','check_replay_attack','verification','emit'])
operation_cost_table

Currently, we disregard the difference in the **AnonParking** `prior_authentication` operation cost due to the unknown reason.

## Verifying the Gas Cost
We summarize the cost of each scheme into the following table:

In [ ]:
initialization_cost = remaining_gas["initialization"]-remaining_gas["beginning"]
execution_cost = operation_cost_table.sum(axis=1)
cost_summary_table = pd.concat([initialization_cost, execution_cost], axis=1, keys=['initialization_cost','execution_cost'])
cost_summary_table['transaction_cost'] = cost_summary_table.sum(axis=1)
cost_summary_table

Comparing this value with the transaction receipt given in the Remix IDE, we concluded that our calculation of operation cost is correct.

From this observation, we concluded that the execution costs vary greatly across the schemes. Both **AnonParking** and **HashAuth** scheme have significantly higher gas cost in their execution cost compared to **PseudoAuth** and our scheme, **NPAuth**. 

Using the operation cost table, we can calculate the operation cost from each scheme. Here we can see that only the `check_replay_attack` operation cost that has significant difference across the scheme. 

To investigate this reason, we analyzed each function that may contribute to a high gas cost in the `checkReplayAttack` function.

## Analyzing the `checkReplayAttack` function cost
In this section, we will analyze the major contributor to the high gas cost in each scheme. As we already explained in the paper, write and read operation plays important role on how to reduce the gas cost in the `checkReplayAttack` function. 

We focused our attention on the following operation due to their high gas cost:
- `SSTORE`: writing operation that costs 20,000 gas or 2,900 gas (it is cheaper to rewrite existing variable)
- `SLOAD` : reading operation that costs 2100 gas (there are `SLOAD` that costs 100 but we classify this `SLOAD` as a minor operation)

We also investigate the hash calculation cost in both **AnonParking** and **HashAuth** scheme to check their contribution in the `checkReplayAttack` function gas cost.

We manually check all of the cost inside the `checkReplayAttack` function from each scheme and summarized our findings as the following:
- **AnonParking** scheme has: 
    - 2 `SSTORE` operations which cost 22,900 gas, 
    - 2 `SLOAD` operations which cost 4,200 gas, and
    - Hash Calculation which costs 472 gas.
- **HashAuth** scheme has:
    - 1 `SSTORE` operation which costs 20,000 gas,
    - 1 `SLOAD` operation which costs 2,100 gas, and
    - Hash Calculation which costs 442 gas.
- **PseudoAuth** scheme has:
    - 1 `SLOAD` operation which costs 2,100 gas.

Our **NPAuth** scheme doesn't have any of these high cost operation, thus has a major difference in its `checkReplayAttack` function gas cost.

We processed these findings into the following table:

In [ ]:
check_replay_attack_cost_table=pd.read_csv("check_replay_attack_cost_table.csv")
check_replay_attack_cost_table_t = check_replay_attack_cost_table.transpose()
check_replay_attack_cost_table_t.columns = check_replay_attack_cost_table_t.iloc[0]
check_replay_attack_cost = check_replay_attack_cost_table_t[1:]
check_replay_attack_cost

To complete our table, we calculate the minor operation gas cost by substracting the `check_replay_attack` gas cost value on the Operation Cost Table with the sum of the check replay attack cost table.

We finished our table with the following calculation:

In [ ]:
major_check_replay_attack_cost= check_replay_attack_cost.sum(axis=1)
minor_operation_cost = operation_cost_table['check_replay_attack']-major_check_replay_attack_cost
minor_operation_cost.name = 'Minor Operation'
total_replay_attack_cost = operation_cost_table['check_replay_attack']
total_replay_attack_cost.name = 'Total Replay Attack Cost'
complete_check_replay_attack_cost_table = pd.concat(
    [
        check_replay_attack_cost,
        minor_operation_cost,
        total_replay_attack_cost
    ]
    ,axis=1)
complete_check_replay_attack_cost_table_t = complete_check_replay_attack_cost_table.transpose()
complete_check_replay_attack_cost_table_t

The previous table is the table that we published on our paper with the name of Replay Attack Prevention Gas Cost Comparison.